In [1]:
import requests
import pandas as pd
import numpy as np
import urllib.request

In [2]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):

    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r *1000

In [3]:
def location_splitter(location):
    location=location.split(' ')
    return location[0],location[1],location[2],location[3]

In [11]:
def roadnamegrabber(latitude,longitude):
    tempResponse=requests.get('https://developers.onemap.sg/privateapi/commonsvc/revgeocode?location=%s,%s&token=%s&buffer=100&addressType=all'%(str(latitude),str(longitude),onemapAPItoken))
    name=pd.DataFrame(eval(tempResponse.content)['GeocodeInfo'])['ROAD'].mode()[0]
    return name.replace('S','s').title()

In [25]:
traffic_image_url='http://datamall2.mytransport.sg/ltaodataservice/Traffic-Imagesv2'
headers_val={'AccountKey':'AO4qMbK3S7CWKSlplQZqlA=='}
traffic_image_req=requests.get(url=traffic_image_url,headers=headers_val)
traffic_image_df=pd.DataFrame(eval(traffic_image_req.content)['value'])

In [26]:
traffic_image_df=traffic_image_df.merge(pd.read_csv('traffic_camera_region.csv',converters={'CameraID':str}),'left','CameraID')

In [6]:
# traffic_speed_url='http://datamall2.mytransport.sg/ltaodataservice/TrafficSpeedBandsv2'
# traffic_speed_req=requests.get(url=traffic_speed_url,headers=headers_val)
# traffic_speed_df=pd.DataFrame(eval(traffic_speed_req.content)['value'])
# road_category_letter=['A','B','C','D','E','F','G']
# road_category_type=['Expressways','Major Arterial Roads','Arterial Roads','Minor Arterial Roads','Small Roads','Slip Roads','No category info available']
# category_mapping=pd.DataFrame({'RoadCategory':road_category_letter,'category':road_category_type})
# traffic_speed_df=traffic_speed_df.merge(category_mapping,'left','RoadCategory')
# traffic_speed_df['start_latitude'],traffic_speed_df['start_longitude'],traffic_speed_df['end_latitude'],traffic_speed_df['end_longitude']=np.vectorize(location_splitter)(traffic_speed_df['Location'])
# traffic_speed_df=traffic_speed_df.drop(['LinkID','RoadCategory','SpeedBand','MaximumSpeed','Location'],axis=1)
# traffic_speed_df=traffic_speed_df.astype({'start_latitude':float,'start_longitude':float,'end_latitude':float,'end_longitude':float})

In [27]:
traffic_incidents_url='http://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents'
traffic_incidents_req=requests.get(url=traffic_incidents_url,headers=headers_val)
traffic_incidents_df=pd.DataFrame(eval(traffic_incidents_req.content)['value'])


# NEA API
For some reason this one need to constantly call and keep track of previous results. Sometimes calls does not return all data.
Some names are given as S123, etc, looking to try OneMap reverse geocoding to get road name from lat long.

In [28]:
weatherreq=requests.get(url='https://api.data.gov.sg/v1/environment/rainfall')
weather_df=pd.DataFrame(eval(weatherreq.content)['metadata']['stations'])

weather_df['latitude']=weather_df['location'].apply(lambda x: x['latitude'])
weather_df['longitude']=weather_df['location'].apply(lambda x: x['longitude'])
weather_df['timestamp']=eval(weatherreq.content)['items'][0]['timestamp']
weather_df['timestamp']=pd.to_datetime(weather_df['timestamp'])

station_rainfall=pd.DataFrame(eval(weatherreq.content)['items'][0]['readings']).rename(columns={'value':'rainfall'})

weather_df=weather_df.merge(station_rainfall,how='left',left_on='id',right_on='station_id')
weather_df=weather_df.drop(['id','device_id','station_id','location'],axis=1)

# Onemap

In [29]:
onemapTokenAPIResponse=requests.post('https://developers.onemap.sg/privateapi/auth/post/getToken',json={'email':'leejin@u.nus.edu','password':'Whysohardtochange123!'})#.content
onemapAPItoken=eval(onemapTokenAPIResponse.content)['access_token']

# Calculations


In [36]:
traffic_image_df['key']=0
traffic_incidents_df['key']=0
weather_df['key']=0

In [38]:
nearest_incidents=traffic_image_df.merge(traffic_incidents_df,'outer','key')
nearest_incidents['incident_distance_from_id']=(np.vectorize(haversine)(nearest_incidents['Latitude_x'],nearest_incidents['Longitude_x'],nearest_incidents['Latitude_y'],nearest_incidents['Longitude_y']))
nearest_incidents=nearest_incidents[nearest_incidents['incident_distance_from_id']<500].sort_values('incident_distance_from_id')
nearest_incidents=nearest_incidents[['CameraID','Message']]

In [43]:
final_df=traffic_image_df.merge(weather_df,'outer','key')
final_df['distance_from_id']=(np.vectorize(haversine)(final_df['Latitude'],final_df['Longitude'],final_df['latitude'],final_df['longitude']))
final_df=final_df.sort_values('distance_from_id').groupby('CameraID').head(1)[['CameraID','Latitude','Longitude','Region','rainfall','ImageLink','Count']]
final_df

,CameraID,Latitude,Longitude,Region,rainfall,ImageLink,Count
853,1701,1.323605,103.858780,Central,0,https://dm-traffic-camera-itsc.s3.ap-southeast...,94
2322,3795,1.301451,103.910596,Central,0,https://dm-traffic-camera-itsc.s3.ap-southeast...,71
2964,4706,1.297920,103.782050,West,0,https://dm-traffic-camera-itsc.s3.ap-southeast...,25
1547,2702,1.445554,103.768340,North,0,https://dm-traffic-camera-itsc.s3.ap-southeast...,17
4810,6713,1.340298,103.945652,East,0,https://dm-traffic-camera-itsc.s3.ap-southeast...,28
...,...,...,...,...,...,...,...
5618,8702,1.390590,103.771700,North,0,https://dm-traffic-camera-itsc.s3.ap-southeast...,11
596,1503,1.270664,103.856978,Central,0,https://dm-traffic-camera-itsc.s3.ap-southeast...,6
1950,2708,1.386500,103.774700,North,0,https://dm-traffic-camera-itsc.s3.ap-southeast...,84
1600,2703,1.350478,103.791034,Central,0,https://dm-traffic-camera-itsc.s3.ap-southeast...,28


In [14]:
nearest_weather=traffic_image_df.merge(weather_df,'outer','key')
nearest_weather['distance_from_id']=(np.vectorize(haversine)(nearest_weather['Latitude'],nearest_weather['Longitude'],nearest_weather['latitude'],nearest_weather['longitude']))
nearest_weather=nearest_weather.sort_values('distance_from_id').groupby('CameraID').head(1).sort_values('distance_from_id')
nearest_weather=nearest_weather

In [15]:
final_df=nearest_weather.merge(nearest_incidents,'left','CameraID')

In [17]:
traffic_image_df

,CameraID,Latitude,Longitude,ImageLink,Region,Count,key
0,1001,1.295313,103.871146,https://dm-traffic-camera-itsc.s3.ap-southeast...,Central,42,0
1,1002,1.319541,103.878563,https://dm-traffic-camera-itsc.s3.ap-southeast...,Central,68,0
2,1003,1.323957,103.872858,https://dm-traffic-camera-itsc.s3.ap-southeast...,Central,37,0
3,1004,1.319536,103.875067,https://dm-traffic-camera-itsc.s3.ap-southeast...,Central,27,0
4,1005,1.363520,103.905394,https://dm-traffic-camera-itsc.s3.ap-southeast...,East,7,0
...,...,...,...,...,...,...,...
82,9702,1.394741,103.817971,https://dm-traffic-camera-itsc.s3.ap-southeast...,North,20,0
83,9703,1.422857,103.773005,https://dm-traffic-camera-itsc.s3.ap-southeast...,North,77,0
84,9704,1.422143,103.795421,https://dm-traffic-camera-itsc.s3.ap-southeast...,North,20,0
85,9705,1.426277,103.787166,https://dm-traffic-camera-itsc.s3.ap-southeast...,North,91,0


In [ ]:
final_df

,CameraID,Type,Message
93,1709,Roadwork,(28/10)20:22 Roadworks on Thomson Road (toward...
292,6703,Roadwork,(28/10)20:34 Roadworks on Thomson Road (toward...
294,6703,Roadwork,(28/10)19:52 Roadworks on Thomson Road (toward...
90,1709,Roadwork,(28/10)20:50 Roadworks on Thomson Road (toward...
290,6703,Roadwork,(28/10)20:50 Roadworks on Thomson Road (toward...
92,1709,Roadwork,(28/10)20:34 Roadworks on Thomson Road (toward...


In [ ]:
final_df.to_csv('main_df.csv',index=False)
nearest_incidents.to_csv('traffic_incidents.csv',index=False)

In [ ]:
final_df


In [ ]:
nearest_incidents.loc[nearest_incidents['CameraID']=='1709']['Message'].to_list()

# END

In [ ]:
df = pd.read_csv('traffic_count_sample.csv')
camera=[]

In [ ]:
camera

In [ ]:
for i in range(len(df)):
    d=dict(name = str(df.iloc[i,0]), lat = df.iloc[i,1], lon = df.iloc[i,2])
    camera.append(d)

In [ ]:
incident_res=''
new_line = '\n'
for i in final_df.loc[final_df['CameraID']=='1004',][['Message']].reset_index(drop=True).itertuples():
    incident_res+= '%s. %s \n'%(str(i[0]+1),i[1])
incident_res

In [ ]:
new_line = '\n'
print(f"Nearby Incidents:{new_line}{new_line.join(incident_res)}")

In [ ]:
final_df[final_df['Message'].notna()]

In [ ]:
final_df.loc[final_df['CameraID']=='1004',][['Message']].itertuples()

In [ ]:
f"{new_line}{new_line.join(incident_res)}"

In [ ]:
# Downloads all the csv files
# traffic_image_df.to_csv('traffic_image.csv',index=False)
# traffic_speed_df.to_csv('traffic_speed.csv',index=False)
# traffic_incidents_df.to_csv('traffic_incidents.csv',index=False)
# weather_df.to_csv('weather.csv',index=False)